In [2]:
from urllib.request import urlretrieve
import pandas as pd


In [39]:
test = urlretrieve(url='https://dd.weather.gc.ca/climate/observations/daily/csv/BC/climate_daily_BC_1010066_1984-09_P1D.csv',filename='Data/climate_daily_BC_1010066_1984-09_P1D.csv')

In [6]:
weather_stations=pd.read_csv('Data/latitude_weathermaps3.csv')

In [7]:
weather_stations.head()

,cluster_label,Station Name,Province,Latitude,Longitude,Elevation,Climate ID,WMO Identifier,TC Identifier,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
0,-1,ABITIBI CAMP 230,ONTARIO,49.35,-89.37,457.2,6040020,71653.0,YAT,1969,1991,1954.0,1994.0,1969.0,1991.0,1969.0,1991.0
1,0,(AE) BOW SUMMIT,ALBERTA,51.70,-116.47,2080.0,3050PPF,71285.0,XAF,1998,2007,1990.0,2024.0,1998.0,2007.0,1998.0,2007.0
2,1,100 MILE HOUSE,BRITISH COLUMBIA,51.65,-121.30,1059.2,1165791,71108.0,YXX,1970,1999,1953.0,2012.0,1970.0,1999.0,1970.0,1999.0
3,2,ABBEY,SASKATCHEWAN,50.70,-108.78,694.9,4020020,NaN,NaN,1956,2008,NaN,NaN,1956.0,2008.0,1956.0,2007.0
4,3,ABERDEEN,SASKATCHEWAN,52.40,-106.17,518.2,4050037,71866.0,YXE,1987,1991,1953.0,2012.0,1987.0,1991.0,1987.0,1991.0


In [9]:
weather_stations.shape

(266, 17)

I'll be using the `weather_stations` table to loop through and download all the data needed from the following site:
https://dd.weather.gc.ca/climate/observations/daily/csv/


To do that cleanly, I will have to make a small adjustment to the way the provinces are currently displayed in the table. We will have to change the full name of the province to their respective 2 letter abbreviation. 

In [10]:
weather_stations['Province'].value_counts()

Province
BRITISH COLUMBIA         50
ONTARIO                  39
QUEBEC                   35
SASKATCHEWAN             34
ALBERTA                  29
MANITOBA                 22
NORTHWEST TERRITORIES    15
NEWFOUNDLAND             14
YUKON TERRITORY          12
NUNAVUT                  11
NEW BRUNSWICK             4
NOVA SCOTIA               1
Name: count, dtype: int64

In [8]:
weather_stations= weather_stations.replace({'Province':{'BRITISH COLUMBIA':'BC','ONTARIO':'ON','QUEBEC':'QC','SASKATCHEWAN':'SK','ALBERTA':'AB','MANITOBA':'MB','NORTHWEST TERRITORIES':'NW','NEWFOUNDLAND':'NL','YUKON TERRITORY':'YK','NUNAVUT':'NU','NEW BRUNSWICK':'NB','NOVA SCOTIA':'NS'}})

In [9]:
weather_stations.head()

,cluster_label,Station Name,Province,Latitude,Longitude,Elevation,Climate ID,WMO Identifier,TC Identifier,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
0,-1,ABITIBI CAMP 230,ON,49.35,-89.37,457.2,6040020,71653.0,YAT,1969,1991,1954.0,1994.0,1969.0,1991.0,1969.0,1991.0
1,0,(AE) BOW SUMMIT,AB,51.70,-116.47,2080.0,3050PPF,71285.0,XAF,1998,2007,1990.0,2024.0,1998.0,2007.0,1998.0,2007.0
2,1,100 MILE HOUSE,BC,51.65,-121.30,1059.2,1165791,71108.0,YXX,1970,1999,1953.0,2012.0,1970.0,1999.0,1970.0,1999.0
3,2,ABBEY,SK,50.70,-108.78,694.9,4020020,NaN,NaN,1956,2008,NaN,NaN,1956.0,2008.0,1956.0,2007.0
4,3,ABERDEEN,SK,52.40,-106.17,518.2,4050037,71866.0,YXE,1987,1991,1953.0,2012.0,1987.0,1991.0,1987.0,1991.0


0      6040020
11     6080192
18     6040325
21     6020LPQ
29     6050NNP
31     6020559
40     6010738
55     6070QK6
66     6041109
81     6061361
96     6061850
97     6071860
101    61219J2
104    6081930
111    6012064
112    6032117
115    6012198
117    6052259
126    6042MJ7
134    6012500
139    6042717
147    6092930
171    6053805
178    604M002
183    6034075
196    6054425
198    6014353
207    6044967
215    6025203
217    6055302
218    6075420
220    6015026
227    6016295
233    6036904
235    6016970
239    6017400
243    6037776
248    6048145
252    6048230
Name: Climate ID, dtype: object

In [3]:

date_range = pd.date_range('01-1990','01-2023',freq='ME').strftime('%Y-%m')

date_range


Index(['1990-01', '1990-02', '1990-03', '1990-04', '1990-05', '1990-06',
       '1990-07', '1990-08', '1990-09', '1990-10',
       ...
       '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08',
       '2022-09', '2022-10', '2022-11', '2022-12'],
      dtype='object', length=396)

In [4]:
weather_data_df=pd.DataFrame()

In [10]:


# downloading weather data
for prov in weather_stations['Province']: #going through each province
    
    station= weather_stations['Climate ID'][weather_stations['Province']== prov] #going through each station from the province
    for id in station:
       
        for date in date_range: #selecting the date range for each province
            
            try:
                download=urlretrieve(url=f'https://dd.weather.gc.ca/climate/observations/daily/csv/{prov}/climate_daily_{prov}_{id}_{date}_P1D.csv',filename=f'Data/Weather/climate_daily_{prov}_{id}_{date}_P1D.csv')
                place_holder = pd.read_csv(f'Data/Weather/climate_daily_{prov}_{id}_{date}_P1D.csv')
                weather_data_df= pd.concat(weather_data_df,place_holder)
                print(f'Downloading from {prov}')
            except:
                continue                 



In [45]:
download=urlretrieve(url='https://dd.weather.gc.ca/climate/observations/daily/csv/ON/climate_daily_ON_6040020_1990-01_P1D.csv',filename=f'Data/test.csv')



HTTPError: HTTP Error 404: Not Found

,cluster_label,Station Name,Province,Latitude,Longitude,Elevation,Climate ID,WMO Identifier,TC Identifier,First Year,Last Year,HLY First Year,HLY Last Year,DLY First Year,DLY Last Year,MLY First Year,MLY Last Year
0,-1,ABITIBI CAMP 230,ON,49.35,-89.37,457.2,6040020,71653.0,YAT,1969,1991,1954.0,1994.0,1969.0,1991.0,1969.0,1991.0
